<a href="https://colab.research.google.com/github/ben854719/DriveSmart-An-Autonomous-Mobility-Solution/blob/main/Diagnostic_Vehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display, HTML
import pandas as pd

# Create a profile of the driver and the vehicle.

df = pd.DataFrame({
    'Name': ['Ben'],
    'Role': ['Driver'],
    'Gender': ['Male'],
    'Nationality': ['Canadian'],
    'Date of Birth': ['1979-02-05'],
    'Social Class': ['Upper Middle Class'],
    'Technology Company': ['Peanut Inc'],
    'Driver License': ['C'],
    'Age': ['46'],
    'Vehicle': ['Mercedes E350'],
    'Vehicle Color': ['Blue'],
    'Self Driving': ['Yes'],
    'Road Assistance': ['Yes'],
    'Road Assistance AI': ['Gemini'],
    'Engine': ['Electric Vehicle'],
    'Tires': ['Goodyear'],
    'Tire Type': ['All season'],
    'Tire Size': ['19 inches'],
    'Transmission': ['Automatic'],
    'Lights': ['LED'],
    'Steering Wheel': ['ABS'],
    'Nightvision': ['Yes'],
    'Temperature detection vehicle': ['Celsius'],
    'Road condition detector': ['Rain, Snow, Fog, Dry'],
    'Bluetooth connection': ['Yes'],
    'Radio Display': ['W214 MBUX'],
    'Plate Number': ['BEN1235'],
    'City': ['Ottawa'],
    'Province': ['Ontario'],
    'Country': ['Canada'],
    'Postal Code': ['K2D 4E5'],
})

display(HTML(df.to_html()))

,Name,Role,Gender,Nationality,Date of Birth,Social Class,Technology Company,Driver License,Age,Vehicle,Vehicle Color,Self Driving,Road Assistance,Road Assistance AI,Engine,Tires,Tire Type,Tire Size,Transmission,Lights,Steering Wheel,Nightvision,Temperature detection vehicle,Road condition detector,Bluetooth connection,Radio Display,Plate Number,City,Province,Country,Postal Code
0,Ben,Driver,Male,Canadian,1979-02-05,Upper Middle Class,Peanut Inc,C,46,Mercedes E350,Blue,Yes,Yes,Gemini,Electric Vehicle,Goodyear,All season,19 inches,Automatic,LED,ABS,Yes,Celsius,"Rain, Snow, Fog, Dry",Yes,W214 MBUX,BEN1235,Ottawa,Ontario,Canada,K2D 4E5


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from IPython.display import clear_output

# Create Syntretic Telemetry Data (Parameters).
num_samples = 1000
num_features = 6 # RPM, voltage, pressure, and temperature.

# Normal data.
normal_data = np.random.normal(loc=0.0, scale=1.0, size=(num_samples, num_features))

# Inject anomalies.
anomaly_indices = np.random.choice(num_samples, size=300, replace=False)
normal_data[anomaly_indices] += np.random.normal(loc=5.0, scale=2.0, size=(300, num_features))

# Convert to Pytorch tensor.
data_tensor = torch.tensor(normal_data, dtype=torch.float32)
dataset = TensorDataset(data_tensor)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Define Autoencoder Model.
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16)
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

# Train the model.
def train(model, dataloader, epochs=20, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        for batch in dataloader:
            inputs = batch[0]
            outputs = model(inputs)
            loss = criterion(outputs, inputs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Detect anomalies.
def detect_anomalies(model, data, threshold=1.5):
  model.eval()
  with torch.no_grad():
    recon = model(data)
    loss = torch.mean((data - recon) ** 2, dim=1)
    anomalies = loss > threshold
    return anomalies, loss

clear_output()
model = Autoencoder(num_features)
train(model, dataloader)

Epoch 1/20, Loss: 9.0032
Epoch 2/20, Loss: 7.2432
Epoch 3/20, Loss: 4.7267
Epoch 4/20, Loss: 2.1653
Epoch 5/20, Loss: 1.6893
Epoch 6/20, Loss: 2.2511
Epoch 7/20, Loss: 1.9679
Epoch 8/20, Loss: 1.7302
Epoch 9/20, Loss: 1.6509
Epoch 10/20, Loss: 1.7353
Epoch 11/20, Loss: 1.5776
Epoch 12/20, Loss: 1.6274
Epoch 13/20, Loss: 1.6562
Epoch 14/20, Loss: 1.6033
Epoch 15/20, Loss: 1.3719
Epoch 16/20, Loss: 1.2616
Epoch 17/20, Loss: 0.9515
Epoch 18/20, Loss: 0.6496
Epoch 19/20, Loss: 0.6590
Epoch 20/20, Loss: 0.4839


In [6]:
!pip install dash dash-core-components dash-html-components plotly --upgrade --no-deps
!pip install flask --upgrade --no-deps
import dash
from dash import dcc, html
import plotly.graph_objects as go

# Data Visualization of the dataset.
app = dash.Dash(__name__)

def create_gauge(title, value, min_val, max_val, thresholds):
    return go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={'text': title},
        gauge={
            'axis': {'range': [min_val, max_val]},
            'steps': [
                {'range': [min_val, thresholds[0]], 'color': "red"},
                {'range': [thresholds[0], thresholds[1]], 'color': "yellow"},
                {'range': [thresholds[1], max_val], 'color': "green"}
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': thresholds[1]
            }
        }
    ))

app.layout = html.Div([
    html.H1("Vehicle System Health Overview", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            dcc.Graph(figure=create_gauge("Engine Temp (°C)", 95, 60, 120, [90, 100]))
        ], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(figure=create_gauge("Battery Voltage (V)", 12.3, 10, 15, [11.5, 12.5]))
        ], style={'width': '33%', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(figure=create_gauge("Brake Pressure (psi)", 32, 0, 50, [20, 35]))
        ], style={'width': '33%', 'display': 'inline-block'}),
    ])
])

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
# Explanation.
The display provides a diagnostic overview of the vehicle’s performance, identifying any anomalies related to its health and operational status.